# tensorboard使用

In [32]:
from torch.utils.tensorboard import SummaryWriter
import torch
import torch.nn as nn

In [3]:
writer = SummaryWriter("training_log")

## 1. 使用 add_scalar 画单条曲线：
图的名字：'y=x平方曲线'  
y坐标的值：i平方  
x坐标的值：i

In [4]:
for i in range(10):
    writer.add_scalar('y=x*x曲线', i*i, i)
writer.close()

然后执行命令，就可以得到网址。

In [7]:
!tensorboard --logdir "training_log"

TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.11.2 at http://localhost:6007/ (Press CTRL+C to quit)
^C


进入网址，就可以看到图片

<img src="../img/y=x*x.png" height="400" width="400" >

## 2. 使用 add_scalars 画多条曲线：
图的名字：'y=x 和 y=2x 曲线'  
y坐标的值：一个字典，对应的是函数的名字，和对应的y坐标值  
x坐标的值：i

In [9]:
for i in range(10):
    writer.add_scalars('y=x 和 y=2x 曲线', {'y=x': i,
                                            'y=2x': i * 2}, i)
writer.close()

<img src="../img/y=x&2x.png" height="400" width="400" >

## 3. 使用 add_graph 构建神经网络结构图：


In [29]:
class SelfAttention(nn.Module):
    def __init__(self, vec_size):
        super().__init__()
        # （Q，K，V）3个w矩阵
        self.QueryW = nn.Linear(vec_size, vec_size)
        self.KeyW = nn.Linear(vec_size, vec_size)
        self.ValueW = nn.Linear(vec_size, vec_size)
        self.Score2Prob = nn.Softmax(dim=-1)
            
    def forward(self, sents):
        '''
        sents: [batch_size, seq_len, vec_size]
        '''
        print('sents', sents.shape)
        # 先计算q，k，v向量
        Q = self.QueryW(sents)  #[batch_size, seq_len, vec_size]
        K = self.KeyW(sents)    #[batch_size, seq_len, vec_size]
        V = self.ValueW(sents)  #[batch_size, seq_len, vec_size]
        # 再计算q和k的相似值，并转成概率分布
        Scores = torch.matmul(Q, K.transpose(1,2).contiguous())  #[batch_size, seq_len, seq_len]
        Scores = self.Score2Prob(Scores)  #[batch_size, seq_len, seq_len]
        # 再输出新的embedding，即：相似值*V
        Out = torch.matmul(Scores,V)  #[batch_size, seq_len, vec_size]
        return Out  
# 实例模型
Model = SelfAttention(vec_size=10)
# 虚构输入
input_sents = torch.rand(2,4,10)
# 输入模型中
Out = Model(input_sents)
print(Out)

sents torch.Size([2, 4, 10])
tensor([[[ 5.3242e-01, -5.3643e-01, -7.0177e-01, -1.9673e-04,  3.1245e-01,
          -3.4367e-02, -6.1487e-04,  2.1513e-01,  4.6962e-01, -2.0229e-01],
         [ 5.4271e-01, -5.3967e-01, -6.9248e-01, -4.3896e-03,  3.0232e-01,
          -3.4397e-02, -2.3759e-03,  2.1602e-01,  4.5965e-01, -2.0195e-01],
         [ 5.3228e-01, -5.3598e-01, -7.0126e-01,  3.3415e-04,  3.1221e-01,
          -3.3858e-02, -8.2075e-04,  2.1505e-01,  4.6933e-01, -2.0229e-01],
         [ 5.4425e-01, -5.5180e-01, -7.0998e-01, -1.3178e-02,  3.1660e-01,
          -4.3668e-02,  8.6894e-03,  2.3284e-01,  4.7298e-01, -1.9724e-01]],

        [[ 4.7209e-01, -4.8592e-01, -6.4677e-01,  1.6429e-01,  3.4273e-01,
          -1.2623e-01,  8.2173e-02,  2.2033e-01,  4.6478e-01, -2.1034e-01],
         [ 4.8148e-01, -4.6062e-01, -6.6268e-01,  2.0381e-01,  3.4542e-01,
          -1.3933e-01,  6.3409e-02,  1.8455e-01,  4.5378e-01, -2.1590e-01],
         [ 4.7790e-01, -4.6042e-01, -6.6239e-01,  2.0162e-01,  

In [31]:
# 画图
writer.add_graph(Model, input_sents)
writer.close()

sents torch.Size([2, 4, 10])
sents torch.Size([2, 4, 10])


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  


sents torch.Size([2, 4, 10])


<img src="../img/model_structure.png" height="400" width="400" >